In [5]:
#vado ad importare le librerie che mi serviranno nello svolgimento del progetto:
import pandas as pd
import numpy as np
from datetime import datetime

In [6]:
#PUNTO 1

In [7]:
#vado a caricare il dataset loan_lenders con pandas, in modo tale da renderlo più facilmente trattabile. 
loan_lenders = pd.read_csv("/Users/matteo/Downloads/loans_lenders.csv")
loan_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [8]:
#devo svolgere la normalizzazione di questo dataframe
#nello split senza mettere str, non me lo farebbe fare, perchè lo considera come una serie
#utilizzo il comando explode(), che mi permette, abbianato a pandas, di trasformare ogni elemento di tipo lista in una riga (row), replicando i valori dell'indice
loan_lenders_normalizzati = loan_lenders.assign(lenders = loan_lenders['lenders'].str.split(',')).explode('lenders')
#così facendo li ho trasformati in tipo string da series che erano, e ho considerato come delimitatore la virgola.

In [9]:
loan_lenders_normalizzati

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499
...,...,...
1387430,1206425,trogdorfamily7622
1387430,1206425,danny6470
1387430,1206425,don6118
1387431,1206486,alan5175


In [10]:
#noto che ci dovrebbero essere degli spazi in lenders, quindi provvedo a toglierli con la funzione replace(), da considerare che opero con una stringa e utilizzerò str.replace()
#questo andrebbe a creare problemi in un momento successivo nella ricerca singola di eventuali lender
loan_lenders_normalizzati['lenders'] = loan_lenders_normalizzati['lenders'].str.replace(" ", "")

In [11]:
loan_lenders_normalizzati

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499
...,...,...
1387430,1206425,trogdorfamily7622
1387430,1206425,danny6470
1387430,1206425,don6118
1387431,1206486,alan5175


In [15]:
#PUNTO 2
#For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [16]:
#carico il dataset loans:
loans = pd.read_csv("/Users/matteo/Downloads/loans.csv")
loans.head()


,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [17]:
#devo trovare la differenza tra disburse time e planned expiration time, andiamo a vedere le variabili che sono raccolte nella tabella:
list(loans)

['loan_id',
 'loan_name',
 'original_language',
 'description',
 'description_translated',
 'funded_amount',
 'loan_amount',
 'status',
 'activity_name',
 'sector_name',
 'loan_use',
 'country_code',
 'country_name',
 'town_name',
 'currency_policy',
 'currency_exchange_coverage_rate',
 'currency',
 'partner_id',
 'posted_time',
 'planned_expiration_time',
 'disburse_time',
 'raised_time',
 'lender_term',
 'num_lenders_total',
 'num_journal_entries',
 'num_bulk_entries',
 'tags',
 'borrower_genders',
 'borrower_pictured',
 'repayment_interval',
 'distribution_model']

In [19]:
#per lo svolgimento della task, mi servono queste due informazioni, ossia quando viene concesso il prestito e quando sarà riscosso:
loans[['disburse_time', 'planned_expiration_time']]

,disburse_time,planned_expiration_time
0,2013-12-22 08:00:00.000 +0000,2014-02-14 03:30:06.000 +0000
1,2013-12-20 08:00:00.000 +0000,2014-03-26 22:25:07.000 +0000
2,2014-01-09 08:00:00.000 +0000,2014-02-15 21:10:05.000 +0000
3,2014-01-17 08:00:00.000 +0000,2014-02-21 03:10:02.000 +0000
4,2013-12-17 08:00:00.000 +0000,2014-02-13 06:10:02.000 +0000
...,...,...
1419602,2015-11-23 08:00:00.000 +0000,2016-01-02 01:00:03.000 +0000
1419603,2015-11-24 08:00:00.000 +0000,2016-01-02 16:40:07.000 +0000
1419604,2015-11-13 08:00:00.000 +0000,2016-01-03 22:20:04.000 +0000
1419605,2015-11-03 08:00:00.000 +0000,2016-01-05 08:50:02.000 +0000


In [18]:
#Utilizzo la libreria datetime per trasformare le date delle due variabili in un formato maneggevole per trovare la differenza in un momento successivo:
#Trasformo quindi in un formato tipico per le date:
loans['disburse_time_date'] = pd.to_datetime(loans['disburse_time'], format= "%Y-%m-%d %H:%M:%S.%f", errors='raise')
loans['planned_expiration_time_date'] = pd.to_datetime(loans['planned_expiration_time'], format= "%Y-%m-%d %H:%M:%S.%f", errors='raise')
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,disburse_time_date,planned_expiration_time_date
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,3,2,1,NaN,female,true,irregular,field_partner,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,11,2,1,NaN,female,true,monthly,field_partner,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,21,2,1,user_favorite,female,true,monthly,field_partner,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00


In [20]:
#Calcolo la differenza in giorni che intercorre:
loans['duration'] = loans['planned_expiration_time_date'] - loans['disburse_time_date']
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,disburse_time_date,planned_expiration_time_date,duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2,1,NaN,female,true,irregular,field_partner,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,53 days 19:30:06
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2,1,NaN,female,true,monthly,field_partner,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,96 days 14:25:07
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,37 days 13:10:05
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2,1,user_favorite,female,true,monthly,field_partner,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,34 days 19:10:02
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,57 days 22:10:02


In [21]:
#Sono presenti vari missing values nelle colonna colonna distribution che sono andato a trovare, provvedo quindi ad eliminare tutti i missing values presenti nella colonna di riferimento
loans.isnull().sum()

loan_id                                 0
loan_name                           47558
original_language                   43251
description                         43285
description_translated             452678
funded_amount                           0
loan_amount                             0
status                                  0
activity_name                           0
sector_name                             0
loan_use                            43273
country_code                            9
country_name                            0
town_name                          161758
currency_policy                         0
currency_exchange_coverage_rate    321526
currency                                0
partner_id                          16790
posted_time                             0
planned_expiration_time            371834
disburse_time                        2813
raised_time                         64282
lender_term                            24
num_lenders_total                 

In [16]:
#andiamo a verificare che entrambe le colonne abbiano presenti dei casi di missing per poi dare la duration come missing:
missing_loans = loans.loc[:,['disburse_time_date','planned_expiration_time_date', 'duration']]
missing_loans.head()

,disburse_time_date,planned_expiration_time_date,duration
0,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,53 days 19:30:06
1,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,96 days 14:25:07
2,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,37 days 13:10:05
3,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,34 days 19:10:02
4,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,57 days 22:10:02


In [17]:
#così facendo posso vedere che effettivamente se un elemento è mancante, allora anche la colonna duration sarà mancante
null_data = missing_loans[missing_loans.isnull().any(axis=1)]
null_data

,disburse_time_date,planned_expiration_time_date,duration
64,2010-11-18 08:00:00+00:00,NaT,NaT
65,2010-11-25 08:00:00+00:00,NaT,NaT
66,2010-11-25 08:00:00+00:00,NaT,NaT
67,2010-11-16 08:00:00+00:00,NaT,NaT
69,2010-11-10 08:00:00+00:00,NaT,NaT
...,...,...,...
1419523,2005-04-14 05:27:55+00:00,NaT,NaT
1419524,NaT,2017-02-09 23:07:54+00:00,NaT
1419525,NaT,2015-09-21 08:06:19+00:00,NaT
1419538,NaT,2017-02-25 20:21:21+00:00,NaT


In [18]:
##Punto 3
#Find the lenders that have funded at least twice.

In [19]:
#devo trovare i landers che hanno finanziato minimo 2 volte:
#ho il dataset normalizzato loan_lenders_normalizzati che contiene i nomi dei lenders, ed ho il dataset loan che contiene tutti i prestiti che effettivamente sono stati finanziati, per prima cosa unisco i due datasets

In [22]:
loan_lenders_normalizzati.head()

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499


In [23]:
#osservo che non ho bisogno di tutto il dataset, ma sono di due colonne che sono l'id del prestito e lo status:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,disburse_time_date,planned_expiration_time_date,duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2,1,NaN,female,true,irregular,field_partner,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,53 days 19:30:06
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2,1,NaN,female,true,monthly,field_partner,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,96 days 14:25:07
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,37 days 13:10:05
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2,1,user_favorite,female,true,monthly,field_partner,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,34 days 19:10:02
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,57 days 22:10:02


In [24]:
#provvedo ad unire con un merge i due dataset sulla base del loan id che viene utilizzato come chiave primaria:
loans_e_lenders_uniti = pd.merge(loan_lenders_normalizzati, loans[['loan_id', 'status']], on = 'loan_id')
loans_e_lenders_uniti

,loan_id,lenders,status
0,483693,muc888,funded
1,483693,sam4326,funded
2,483693,camaran3922,funded
3,483693,lachheb1865,funded
4,483693,rebecca3499,funded
...,...,...,...
28293907,1206425,trogdorfamily7622,funded
28293908,1206425,danny6470,funded
28293909,1206425,don6118,funded
28293910,1206486,alan5175,funded


In [25]:
#mi rendo conto che nello status non ho tutti con esito positivo, che sarebbe funded, quindi per non rendere troppo complicato il comando del count successivo, considero solamente i casi di funded:
loans_e_lenders_uniti.status.unique()

array(['funded', 'expired', 'refunded', 'fundRaising'], dtype=object)

In [26]:
#scelgo solamente i loans con esito positivo quindi:
loans_e_lenders_uniti_funded = loans_e_lenders_uniti[loans_e_lenders_uniti['status'] == 'funded']
loans_e_lenders_uniti_funded.head()

,loan_id,lenders,status
0,483693,muc888,funded
1,483693,sam4326,funded
2,483693,camaran3922,funded
3,483693,lachheb1865,funded
4,483693,rebecca3499,funded


In [28]:
#Eseguo un groupby per evidenziare per ogni lenders, quante volte si presentano all'interno del dataset:
loans_e_lenders_uniti_funded_1= loans_e_lenders_uniti_funded.groupby('lenders')['lenders'].size()
loans_e_lenders_uniti_funded_1

lenders
000                 39
00000               38
0002                70
00mike00             1
0101craign0101      71
                  ... 
zzmcfate          1648
zzpaghetti9994       1
zzrvmf8538          13
zzzsai               4
zzzworld             1
Name: lenders, Length: 1368261, dtype: int64

In [29]:
#mettendo come minimo uguale a 2, trovo i lenders che hanno finanziato con successo minimo due volte:
loans_e_lenders_uniti_funded_1[loans_e_lenders_uniti_funded_1 >= 2]

lenders
000                 39
00000               38
0002                70
0101craign0101      71
0132575              4
                  ... 
zzinnia             27
zzivnk7094           4
zzmcfate          1648
zzrvmf8538          13
zzzsai               4
Name: lenders, Length: 893981, dtype: int64

In [30]:
##Punto 4
#For each country, compute how many loans have involved that country as borrowers.
#Per ciascun paese, calcolare quanti prestiti hanno coinvolto quel paese come mutuatari

In [31]:
#carico il dataset con le informazioni legati ai vari luoghi geografici:
locations = pd.read_csv("/Users/matteo/Downloads/locations.csv")
locations.head()

,address_components,formatted_address,geometry.bounds.northeast.lat,geometry.bounds.northeast.lng,geometry.bounds.southwest.lat,geometry.bounds.southwest.lng,geometry.location.lat,geometry.location.lng,geometry.location_type,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,location_name,partial_match,place_id,types
0,"[{'long_name': 'Managua', 'short_name': 'Manag...","Managua, Nicaragua",12.175357,-86.125088,12.019961,-86.391833,12.114993,-86.236174,APPROXIMATE,12.175357,-86.125088,12.019961,-86.391833,"Managua, Nicaragua",NaN,ChIJC4gH2Q1WcY8RakxPFKL0Xmw,"['locality', 'political']"
1,"[{'long_name': 'Lahore', 'short_name': 'Lahore...","Lahore, Pakistan",31.698486,74.587397,31.267394,74.115539,31.520370,74.358747,APPROXIMATE,31.698486,74.587397,31.267394,74.115539,"Lahore, Pakistan",NaN,ChIJ2QeB5YMEGTkRYiR-zGy-OsI,"['locality', 'political']"
2,"[{'long_name': 'Kaduna', 'short_name': 'Kaduna...","Kaduna, Nigeria",10.616996,7.508812,10.397157,7.349789,10.510464,7.416505,APPROXIMATE,10.616996,7.508812,10.397157,7.349789,"Kaduna, Nigeria",NaN,ChIJdRc3NFg1TRARdOG_mpeVAUg,"['locality', 'political']"
3,"[{'long_name': 'Cusco', 'short_name': 'Cusco',...","Cusco, Peru",-13.497391,-71.853332,-13.562648,-72.025166,-13.531950,-71.967463,APPROXIMATE,-13.497391,-71.853332,-13.562648,-72.025166,"Cusco, Peru",NaN,ChIJMYRZJtjVbZERXTEYI8yWqSo,"['locality', 'political']"
4,"[{'long_name': 'León', 'short_name': 'León', '...","León, Nicaragua",12.460966,-86.843405,12.407026,-86.920753,12.431553,-86.872215,APPROXIMATE,12.460966,-86.843405,12.407026,-86.920753,"Leon, Nicaragua",NaN,ChIJd3qPu5IfcY8RPDHqZG_AtTo,"['locality', 'political']"


In [32]:
#Osservo i vari nomi delle nazioni:
loans['country_name']

0          Philippines
1             Honduras
2             Pakistan
3           Kyrgyzstan
4          Philippines
              ...     
1419602          Kenya
1419603    Philippines
1419604          Ghana
1419605    Philippines
1419606     Madagascar
Name: country_name, Length: 1419607, dtype: object

In [33]:
#sapendo che prima ci sono stati dei casi dove la duration è stata nulla, considero solamente i prestiti dove la durata è stata calcolata:
loans_nomissing = loans[loans.duration.notnull()]

In [37]:
#eseguo il groupby per tutto per trovare il numero di country
loans_nazioni = loans_nomissing.groupby('country_name').size()
loans_nazioni.head(10)

country_name
Afghanistan        2
Albania         3075
Armenia        12809
Azerbaijan      4026
Belize           218
Benin           2452
Bhutan             2
Bolivia        15515
Botswana           1
Brazil           482
dtype: int64

In [41]:
#Punto 5
#For each country, compute the overall amount of money borrowed.

In [42]:
#osservo dalla serie di informazioni del dataset loans, che è presenta una colonna chiamata loan_amount che rappresenta l'ammontare di denaro concesso in prestito
list(loans)

['loan_id',
 'loan_name',
 'original_language',
 'description',
 'description_translated',
 'funded_amount',
 'loan_amount',
 'status',
 'activity_name',
 'sector_name',
 'loan_use',
 'country_code',
 'country_name',
 'town_name',
 'currency_policy',
 'currency_exchange_coverage_rate',
 'currency',
 'partner_id',
 'posted_time',
 'planned_expiration_time',
 'disburse_time',
 'raised_time',
 'lender_term',
 'num_lenders_total',
 'num_journal_entries',
 'num_bulk_entries',
 'tags',
 'borrower_genders',
 'borrower_pictured',
 'repayment_interval',
 'distribution_model',
 'disburse_time_date',
 'planned_expiration_time_date',
 'duration']

In [43]:
#eseguo un groupby per trovare per ogni nazione l'importo di denaro preso in prestito, applicando un sum() per ottenere il totale:
loans_amount = loans_nomissing.groupby('country_code')['loan_amount'].sum()
loans_amount

country_code
AF       14000.0
AL     4307350.0
AM    20579275.0
AZ     6839400.0
BF     3990450.0
         ...    
XK     3083025.0
YE     3430100.0
ZA      886150.0
ZM     1978975.0
ZW     4921625.0
Name: loan_amount, Length: 89, dtype: float64

In [44]:
#Punto 6
#Like the previous point, but expressed as a percentage of the overall amount lent.

In [45]:
#dobbiamo calcolare come il punto precedente ma come percentuale del totale dell'ammonatare totale del prestito:

In [46]:
#Per prima cosa calcolo il totale dei prestiti:
loans_tot = loans_nomissing['loan_amount'].sum()
loans_tot

889804100.0

In [47]:
#Per prima cosa vado a resettare il mio index in modo tale da permettermi di selezionare successivamente le colonne che mi interessano:
loans_amount_percentuale = loans_nomissing.groupby('country_name')['loan_amount'].sum().reset_index()
#adesso calcolo la percentuale, utilizzando il totale dei prestiti che ho calcolato precedentemente:
loans_amount_percentuale['loan_amount'] = loans_amount_percentuale['loan_amount']/loans_tot*100
loans_amount_percentuale

,country_name,loan_amount
0,Afghanistan,0.001573
1,Albania,0.484078
2,Armenia,2.312787
3,Azerbaijan,0.768641
4,Belize,0.016877
...,...,...
85,Vanuatu,0.001040
86,Vietnam,2.303319
87,Yemen,0.385489
88,Zambia,0.222406


In [48]:
#Punto 7
#Like the three previous points, but split for each year (with respect to disburse time).

In [49]:
#dobbiamo eseguire i punti precedenti ma legati ad ogni singolo anno su disburse_time.
#la prima cosa che devo fare è lavorare su disburse_time ottenere ogni singolo anno.


In [50]:
#Utilizzo loans_nomissing poichè considero sempre i prestiti che non hanno valori nulli:
#come nel punto due, devo trasformare in un formato datetime le colonne di mio interesse:
loans_nomissing["disburse_time_new"] = pd.to_datetime(loans_nomissing["disburse_time"])
#estraggo l'anno al quale è riferito disburse_time_new:
loans_nomissing["year"] = loans_nomissing["disburse_time_new"].dt.year
loans_nomissing

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,disburse_time_date,planned_expiration_time_date,duration,disburse_time_new,year
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,NaN,female,true,irregular,field_partner,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,53 days 19:30:06,2013-12-22 08:00:00+00:00,2013
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,NaN,female,true,monthly,field_partner,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,96 days 14:25:07,2013-12-20 08:00:00+00:00,2013
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,37 days 13:10:05,2014-01-09 08:00:00+00:00,2014
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,user_favorite,female,true,monthly,field_partner,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,34 days 19:10:02,2014-01-17 08:00:00+00:00,2014
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,57 days 22:10:02,2013-12-17 08:00:00+00:00,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419602,988180,NaN,NaN,NaN,NaN,400.0,400.0,funded,Tailoring,Services,...,"#Parent, #Repeat Borrower, #Woman Owned Biz",NaN,NaN,monthly,field_partner,2015-11-23 08:00:00+00:00,2016-01-02 01:00:03+00:00,39 days 17:00:03,2015-11-23 08:00:00+00:00,2015
1419603,988213,Perlita,English,"Perlita is 52 years old, married and has three...","Perlita is 52 years old, married and has three...",300.0,300.0,funded,Pigs,Agriculture,...,"#Animals, #Elderly, #Repeat Borrower, #Woman O...",female,true,irregular,field_partner,2015-11-24 08:00:00+00:00,2016-01-02 16:40:07+00:00,39 days 08:40:07,2015-11-24 08:00:00+00:00,2015
1419604,989109,Okyeso Nyame Group,English,Okyeso Nyame group will begin its third cycle ...,Okyeso Nyame group will begin its third cycle ...,2425.0,2425.0,funded,Bakery,Food,...,"user_favorite, #Parent, #Vegan, #Woman Owned B...","female, female, female, male, male, female","true, true, true, true, true, true",irregular,field_partner,2015-11-13 08:00:00+00:00,2016-01-03 22:20:04+00:00,51 days 14:20:04,2015-11-13 08:00:00+00:00,2015
1419605,989143,Exequila,English,"Exequila is from San Miguel, Bohol. She is in...","Exequila is from San Miguel, Bohol. She is in...",100.0,100.0,funded,Farming,Agriculture,...,NaN,female,true,irregular,field_partner,2015-11-03 08:00:00+00:00,2016-01-05 08:50:02+00:00,63 days 00:50:02,2015-11-03 08:00:00+00:00,2015


In [51]:
#Eseguendo un groupby basato sul nome della nazione e sull'anno, trovo la dimensione dei prestiti per nazioni e per anno:
loans_nomissing.groupby(['country_name', 'year']).size()

country_name  year
Afghanistan   2015       1
              2016       1
Albania       2012     332
              2013     507
              2014     603
                      ... 
Zimbabwe      2013     426
              2014    2078
              2015     600
              2016     808
              2017    1079
Length: 516, dtype: int64

In [52]:
#Ora, seguendo la falsa riga del passo precedente, prendo in considerazione la somma dell'ammontare dei prestiti per nazione e per anno: 
loans_nomissing.groupby(['country_name', 'year'])['loan_amount'].sum()

country_name  year
Afghanistan   2015       6000.0
              2016       8000.0
Albania       2012     413000.0
              2013     786950.0
              2014     902550.0
                        ...    
Zimbabwe      2013     678525.0
              2014    1311575.0
              2015     723625.0
              2016     788600.0
              2017    1237600.0
Name: loan_amount, Length: 516, dtype: float64

In [53]:
#calcolo la percentuale di prestiti sul totale:
loans_nomissing.groupby(['country_name', 'year'])['loan_amount'].sum()/loans_tot*100

country_name  year
Afghanistan   2015    0.000674
              2016    0.000899
Albania       2012    0.046415
              2013    0.088441
              2014    0.101432
                        ...   
Zimbabwe      2013    0.076256
              2014    0.147400
              2015    0.081324
              2016    0.088626
              2017    0.139087
Name: loan_amount, Length: 516, dtype: float64

In [54]:
#Punto 8
#For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.
#per ogni lenders trovare l'ammontare di denaro prestato, se un prestito fosse stato finanziato da più lenders allora sono da considerare di pari importo i finanziamenti dei singoli lenders

In [56]:
#vado a contare quanti lenders sono coinvolti per ogni loan, avendo diviso nel primo punto i lenders per ogni singolo loan, eseguo:
lenders_count = loan_lenders_normalizzati.groupby('loan_id').count()
lenders_count

,lenders
loan_id,
84,3
85,2
86,3
88,3
89,4
...,...
1444051,1
1444053,1
1444058,1


In [57]:
#eseguo un merge per unire i due dataset, sulla base del loan_id uguale per i due datasets:
lenders_count_e_amout = pd.merge(lenders_count, loans_nomissing, on = 'loan_id')
lenders_count_e_amout[['loan_id', 'lenders', 'loan_amount']].head()

,loan_id,lenders,loan_amount
0,358219,74,2300.0
1,358926,60,1925.0
2,359566,42,1400.0
3,360000,90,2875.0
4,360232,58,1800.0


In [58]:
#ora devo dividere l'ammontare del prestito per il numero di lenders che sono presenti e che lo hanno finanziato
lenders_count_e_amout['amount_unitario'] = lenders_count_e_amout['loan_amount']/lenders_count_e_amout['lenders']
lenders_count_e_amout.head()

,loan_id,lenders,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,...,borrower_genders,borrower_pictured,repayment_interval,distribution_model,disburse_time_date,planned_expiration_time_date,duration,disburse_time_new,year,amount_unitario
0,358219,74,Mirismayil,English,Mirismayil is a 28-year-old trader who sells d...,NaN,2300.0,2300.0,funded,Clothing Sales,...,male,true,monthly,field_partner,2011-10-18 07:00:00+00:00,2012-02-11 22:50:58+00:00,116 days 15:50:58,2011-10-18 07:00:00+00:00,2011,31.081081
1,358926,60,Saiq,English,"Saiq, 45, sells men's and women's clothes in t...",NaN,1925.0,1925.0,funded,Clothing,...,male,true,monthly,field_partner,2011-10-19 07:00:00+00:00,2012-01-30 08:01:57+00:00,103 days 01:01:57,2011-10-19 07:00:00+00:00,2011,32.083333
2,359566,42,Alikishi,English,Alikishi is a 59-year-old man who lives in Saa...,NaN,1400.0,1400.0,funded,Grocery Store,...,male,true,monthly,field_partner,2011-10-19 07:00:00+00:00,2012-02-09 01:35:52+00:00,112 days 18:35:52,2011-10-19 07:00:00+00:00,2011,33.333333
3,360000,90,Dagva,English,Dagva is 31 years old and lives with his wife ...,NaN,2875.0,2875.0,funded,Transportation,...,male,true,monthly,field_partner,2011-11-15 08:00:00+00:00,2012-02-03 21:30:05+00:00,80 days 13:30:05,2011-11-15 08:00:00+00:00,2011,31.944444
4,360232,58,Amil,English,"Amil is 38 years old, married and has two sons...",NaN,1800.0,1800.0,funded,General Store,...,male,true,monthly,field_partner,2011-10-21 07:00:00+00:00,2012-02-09 01:35:52+00:00,110 days 18:35:52,2011-10-21 07:00:00+00:00,2011,31.034483


In [59]:
#avendo trovato l'ammontare per ogni singolo prestito, allora passo ad aggiungere il nome dei lenders per trovare per ogni lenders quanto effettivamente ha contribuito
lenders_e_amount_loans = pd.merge(lenders_count_e_amout, loan_lenders_normalizzati, on = 'loan_id', how = 'right')
lenders_e_amount_loans.head()

,loan_id,lenders_x,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,...,borrower_pictured,repayment_interval,distribution_model,disburse_time_date,planned_expiration_time_date,duration,disburse_time_new,year,amount_unitario,lenders_y
0,358219,74.0,Mirismayil,English,Mirismayil is a 28-year-old trader who sells d...,NaN,2300.0,2300.0,funded,Clothing Sales,...,true,monthly,field_partner,2011-10-18 07:00:00+00:00,2012-02-11 22:50:58+00:00,116 days 15:50:58,2011-10-18 07:00:00+00:00,2011.0,31.081081,james61968508
1,358219,74.0,Mirismayil,English,Mirismayil is a 28-year-old trader who sells d...,NaN,2300.0,2300.0,funded,Clothing Sales,...,true,monthly,field_partner,2011-10-18 07:00:00+00:00,2012-02-11 22:50:58+00:00,116 days 15:50:58,2011-10-18 07:00:00+00:00,2011.0,31.081081,james61968508
2,358219,74.0,Mirismayil,English,Mirismayil is a 28-year-old trader who sells d...,NaN,2300.0,2300.0,funded,Clothing Sales,...,true,monthly,field_partner,2011-10-18 07:00:00+00:00,2012-02-11 22:50:58+00:00,116 days 15:50:58,2011-10-18 07:00:00+00:00,2011.0,31.081081,james61968508
3,358219,74.0,Mirismayil,English,Mirismayil is a 28-year-old trader who sells d...,NaN,2300.0,2300.0,funded,Clothing Sales,...,true,monthly,field_partner,2011-10-18 07:00:00+00:00,2012-02-11 22:50:58+00:00,116 days 15:50:58,2011-10-18 07:00:00+00:00,2011.0,31.081081,TippieMBAUIowaGFMA
4,358219,74.0,Mirismayil,English,Mirismayil is a 28-year-old trader who sells d...,NaN,2300.0,2300.0,funded,Clothing Sales,...,true,monthly,field_partner,2011-10-18 07:00:00+00:00,2012-02-11 22:50:58+00:00,116 days 15:50:58,2011-10-18 07:00:00+00:00,2011.0,31.081081,tom1045


In [60]:
#ora non mi resta che effettuare un groupby per evidenziare per ogni lenders l'import unitario che è stato da loro sottoscritto:
lenders_ammontare = lenders_e_amount_loans.groupby("lenders_y")["amount_unitario"].sum().reset_index()
lenders_ammontare.head()

,lenders_y,amount_unitario
0,000,1456.996604
1,00000,0.000000
2,0002,677.081981
3,00mike00,52.631579
4,0101craign0101,606.748996


In [61]:
#Punto 9
#For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [62]:
list(loans)

['loan_id',
 'loan_name',
 'original_language',
 'description',
 'description_translated',
 'funded_amount',
 'loan_amount',
 'status',
 'activity_name',
 'sector_name',
 'loan_use',
 'country_code',
 'country_name',
 'town_name',
 'currency_policy',
 'currency_exchange_coverage_rate',
 'currency',
 'partner_id',
 'posted_time',
 'planned_expiration_time',
 'disburse_time',
 'raised_time',
 'lender_term',
 'num_lenders_total',
 'num_journal_entries',
 'num_bulk_entries',
 'tags',
 'borrower_genders',
 'borrower_pictured',
 'repayment_interval',
 'distribution_model',
 'disburse_time_date',
 'planned_expiration_time_date',
 'duration']

In [63]:
#per prima cosa ho bisogno di sapere da dove provengono i prestiti a livello di nazioni:
lenders = pd.read_csv('/Users/matteo/Downloads/lenders.csv')
lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


In [64]:
#mi rendo conto che nella colonna country_code trovo il codice di ogni nazione riferita ad ogni singolo lender
lenders['country_code'].unique()

array([nan, 'US', 'IN', 'AU', 'PL', 'BR', 'CN', 'FR', 'BG', 'AT', 'GB',
       'ES', 'DE', 'BE', 'SK', 'IT', 'SE', 'JP', 'PK', 'LA', 'AR', 'HU',
       'TR', 'LV', 'TW', 'MY', 'NL', 'GE', 'ID', 'RU', 'CA', 'PT', 'NG',
       'IS', 'NO', 'HK', 'JO', 'KE', 'MT', 'VN', 'CH', 'NZ', 'FI', 'MX',
       'TN', 'CO', 'AE', 'KR', 'DK', 'TZ', 'PH', 'GR', 'RO', 'CR', 'IL',
       'SA', 'GH', 'PR', 'VE', 'LK', 'EG', 'MA', 'TH', 'AM', 'PG', 'CZ',
       'SG', 'CL', 'YE', 'CY', 'PY', 'HR', 'MN', 'CM', 'UG', 'LT', 'EE',
       'BH', 'DZ', 'IE', 'RS', 'KY', 'IR', 'GT', 'DO', 'PS', 'PE', 'BD',
       'ET', 'SY', 'UY', 'RW', 'AS', 'PA', 'AL', 'UA', 'EC', 'KW', 'AD',
       'NI', 'ZA', 'MM', 'FJ', 'LC', 'IQ', 'HT', 'AO', 'SD', 'SI', 'MK',
       'OM', 'AQ', 'LY', 'AZ', 'AF', 'MO', 'BS', 'MZ', 'SV', 'ZM', 'LU',
       'HN', 'BI', 'BJ', 'YT', 'GU', 'ML', 'BO', 'GW', 'TJ', 'KH', 'SR',
       'LB', 'VU', 'AN', 'MP', 'MW', 'QA', 'MH', 'TM', 'UM', 'SZ', 'XK',
       'NE', 'TT', 'SO', 'ZW', 'TK', 'BB', 'JM', 'NP

In [65]:
#vediamo quanti valori mancanti sono presenti nella colonna di country_name:
lenders['country_code'].isnull().sum()

1458635

In [66]:
#di questa serie di informazioni quelle che ci servono, sono il nome dei lenders ed il country_code, non avrebbe senso tenere tutto il dataset:
lenders_infoutili = lenders[['permanent_name', 'country_code']]
lenders_infoutili.head()

,permanent_name,country_code
0,qian3013,NaN
1,reena6733,NaN
2,mai5982,NaN
3,andrew86079135,NaN
4,nguyen6962,NaN


In [67]:
#visto che per i dati mancanti devo sostituire cercando di impartire i dati mancanti seguendo la distribuzione di quelli noti:
#per prima cosa capisco la distribuzione di quelli noti volendo:
lenders_conteggio = lenders_infoutili.groupby('country_code').count().reset_index().rename(columns={'permanent_name':'conteggio_country'})
lenders_conteggio

,country_code,conteggio_country
0,AD,15
1,AE,1043
2,AF,228
3,AG,8
4,AI,4
...,...,...
229,YE,195
230,YT,2
231,ZA,1051
232,ZM,65


In [68]:
#calcolo il numero di lenders presenti come informazioni:
len(lenders)

2349174

In [69]:
#trovo il numero di country segnate per i lenders
country_segnate = 2349174 - 1458635
country_segnate

890539

In [70]:
#capisco quanto rappresentano della percentuale del totale i vari lenders
lenders_conteggio['percentuale'] = lenders_conteggio['conteggio_country']/ country_segnate*100
lenders_conteggio

,country_code,conteggio_country,percentuale
0,AD,15,0.001684
1,AE,1043,0.117120
2,AF,228,0.025602
3,AG,8,0.000898
4,AI,4,0.000449
...,...,...,...
229,YE,195,0.021897
230,YT,2,0.000225
231,ZA,1051,0.118018
232,ZM,65,0.007299


In [71]:
#trovo la lista delle percentuali, questo mi sarà utili in un momento successivo, poichè utilizzerò la lista per impartire su tutto il dataset, dove i miei dati non sono noti, i valori che mi interesserean nelle varie proporzioni:
lista_percentuale = lenders_conteggio['percentuale'].to_numpy()
#trovo la lista delle nazioni, inteso come sigle di riconoscimento, per poi posizionarle seguendo la percentuale, all'interno dell'interno datasets:
codici_country = lenders_conteggio['country_code'].to_numpy()
#calcolo la percentuale assoluta per ogni singola country:
lenders_conteggio['percentuale_unitaria'] = lenders_conteggio['percentuale'] / lenders_conteggio['percentuale'].sum()
#trasformo questa lista di percentuali assolute in una lista, in modo tale da utilizzarla per posizionare all'interno del dataset le varie country che non sono rese note:
lista_percentuale_unitaria = lenders_conteggio['percentuale_unitaria'].to_numpy()

In [72]:
#Utilizzo questo comando per andare a posizionare all'interno della colonna che non conteneva le nazioni come conosciute, le varie nazioni in base alla percentuale sulla base della lista che ho creato in maniera precedente:
lenders['country_code'] = lenders['country_code'].fillna(pd.Series(np.random.choice(codici_country, p= lista_percentuale_unitaria, size=len(lenders))))
lenders.head(20)

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,US,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,CA,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,US,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,FR,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,LA,1461301154,NaN,NaN,NaN,NaN,0
5,sirinapa6764,Sirinapa,NaN,NaN,US,1461301496,NaN,NaN,1.0,NaN,0
6,rene7585,Rene,NaN,NaN,US,1461301636,NaN,NaN,2.0,NaN,0
7,harald2826,Harald,NaN,NaN,US,1461301670,NaN,NaN,2.0,NaN,0
8,mehdi2903,Mehdi,NaN,NaN,BR,1461301756,NaN,NaN,NaN,NaN,0
9,youchan8125,Youchan,NaN,NaN,US,1461301941,NaN,NaN,1.0,NaN,0


In [73]:
#vado a riprendere la tabella dell'ammontare unitario per ogni lenders:
lenders_ammontare.head()

,lenders_y,amount_unitario
0,000,1456.996604
1,00000,0.000000
2,0002,677.081981
3,00mike00,52.631579
4,0101craign0101,606.748996


In [74]:
#vado ad unire l'ammontare unitario per ogni singolo lenders, con le informazioni sui lenders:
lenders_semi = pd.merge(lenders, lenders_ammontare, left_on = 'permanent_name', right_on = 'lenders_y')
lenders_semi.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited,lenders_y,amount_unitario
0,qian3013,Qian,NaN,NaN,US,1461300457,NaN,NaN,1.0,NaN,0,qian3013,46.839080
1,reena6733,Reena,NaN,NaN,CA,1461300634,NaN,NaN,9.0,NaN,0,reena6733,417.948366
2,andrew86079135,Andrew,NaN,NaN,FR,1461301091,NaN,NaN,5.0,Peter Tan,0,andrew86079135,349.601088
3,sirinapa6764,Sirinapa,NaN,NaN,US,1461301496,NaN,NaN,1.0,NaN,0,sirinapa6764,50.000000
4,rene7585,Rene,NaN,NaN,US,1461301636,NaN,NaN,2.0,NaN,0,rene7585,59.126984


In [75]:
#ora che i vari lenders per ogni stato, non mi resta che raggruppare gli stati in modo tale da ottenere la somma di ogni ammontare devoluto da ogni stato:
#arrotondo il risultato a nessuna cifra decimale:
lenders_semi_denaro = round(lenders_semi.groupby('country_code')['amount_unitario'].sum(),0)
lenders_semi_denaro.head(20)

country_code
AD       13372.0
AE     1593650.0
AF      155308.0
AG        1077.0
AI         699.0
AL       38769.0
AM       25717.0
AN       31992.0
AO       32751.0
AQ       21405.0
AR     1131475.0
AS       11213.0
AT     2318724.0
AU    50008545.0
AW        7636.0
AZ        7758.0
BA       57436.0
BB       19598.0
BD       68814.0
BE     2643807.0
Name: amount_unitario, dtype: float64

In [76]:
#osservo quindi il valore del loan per ogni prestito:
loans_amount.head(20)

country_code
AF       14000.0
AL     4307350.0
AM    20579275.0
AZ     6839400.0
BF     3990450.0
BI     4476075.0
BJ     2132125.0
BO    31839450.0
BR     1192325.0
BT       20000.0
BW        8000.0
BZ      150175.0
CA       50000.0
CD    16363725.0
CG     1943625.0
CI      119425.0
CL     1691650.0
CM     1815825.0
CN      380525.0
CO    21614250.0
Name: loan_amount, dtype: float64

In [78]:
#calcolo la differenza tra l'ammontare dei lenders e l'ammontare dei prestiti:
denaro_differenza = lenders_semi_denaro - loans_amount
#elimino dalla diffenza i casi in cui il denaro risultato come valore mancante, perchè è un'informazioni che non ci è utile:
differenza_nomissing = denaro_differenza.dropna()
differenza_nomissing
differenza_nomissing.head(20)

country_code
AF      141308.0
AL    -4268581.0
AM   -20553558.0
AZ    -6831642.0
BF    -3986405.0
BI    -4474251.0
BJ    -2127195.0
BO   -31807965.0
BR      904295.0
BT      -17422.0
BW       -2121.0
BZ     -129986.0
CA    68982039.0
CD   -16355228.0
CG    -1939200.0
CI     -101264.0
CL    -1430347.0
CM    -1791208.0
CN      664502.0
CO   -21308121.0
dtype: float64

In [79]:
#vado a creare quindi un dataframe riepilogativo che mi permette di osservare meglio l'andamento dell'ammmontare della differenza per ogni singola country:
differenza = {"differenza_nomissing": differenza_nomissing}
differenza_tabella = pd.DataFrame(differenza)
differenza_tabella.reset_index()

,country_code,differenza_nomissing
0,AF,141308.0
1,AL,-4268581.0
2,AM,-20553558.0
3,AZ,-6831642.0
4,BF,-3986405.0
...,...,...
84,XK,-3079133.0
85,YE,-3356887.0
86,ZA,-319668.0
87,ZM,-1940752.0


In [80]:
#Punto 10
#Which country has the highest ratio between the difference computed at the previous point and the population?

In [81]:
#vado a caricare le statistiche per ogni nazione:
statistiche_country = pd.read_csv('/Users/matteo/Downloads/country_stats.csv')
statistiche_country.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh


In [82]:
#uniamo i datasets, sulla base della sigla della nazione:
statistiche_unite = pd.merge(statistiche_country, differenza_tabella, left_on = 'country_code', right_on = 'country_code')
statistiche_unite.head(10)

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,differenza_nomissing
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India,-6832668.0
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria,-2390881.0
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico,-14645955.0
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan,-19823359.0
4,Democratic Republic of the Congo,CD,COD,Africa,Middle Africa,81339988,63.0,0.435465,59.057,9.750000,6.117153,679.631223,The Democratic Republic of the Congo,-16355228.0
5,United States,US,USA,Americas,Northern America,324459463,15.1,0.919553,79.222,16.538150,13.217890,53245.076960,United States,499188329.0
6,China,CN,CHN,Asia,Eastern Asia,1409517397,3.3,0.737681,75.963,13.535750,7.641840,13345.477460,China,664502.0
7,Indonesia,ID,IDN,Asia,South-eastern Asia,263991379,10.9,0.688847,69.052,12.867660,7.934817,10053.337700,Indonesia,-5938839.0
8,Egypt,EG,EGY,Africa,Northern Africa,97553151,25.2,0.691392,71.325,13.098900,7.110000,10064.059910,Egypt,-1195453.0
9,Philippines,PH,PHL,Asia,South-eastern Asia,104918090,21.6,0.681799,68.340,11.733020,9.326870,8395.094758,Philippines,-80639361.0


In [83]:
#calcolo la differenza della popolazione:
statistiche_unite['differenza_popolazione'] = statistiche_unite['differenza_nomissing'] / statistiche_unite['population']
statistiche_unite.head(10)

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,differenza_nomissing,differenza_popolazione
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India,-6832668.0,-0.005102
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria,-2390881.0,-0.012525
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico,-14645955.0,-0.113391
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan,-19823359.0,-0.100618
4,Democratic Republic of the Congo,CD,COD,Africa,Middle Africa,81339988,63.0,0.435465,59.057,9.750000,6.117153,679.631223,The Democratic Republic of the Congo,-16355228.0,-0.201072
5,United States,US,USA,Americas,Northern America,324459463,15.1,0.919553,79.222,16.538150,13.217890,53245.076960,United States,499188329.0,1.538523
6,China,CN,CHN,Asia,Eastern Asia,1409517397,3.3,0.737681,75.963,13.535750,7.641840,13345.477460,China,664502.0,0.000471
7,Indonesia,ID,IDN,Asia,South-eastern Asia,263991379,10.9,0.688847,69.052,12.867660,7.934817,10053.337700,Indonesia,-5938839.0,-0.022496
8,Egypt,EG,EGY,Africa,Northern Africa,97553151,25.2,0.691392,71.325,13.098900,7.110000,10064.059910,Egypt,-1195453.0,-0.012254
9,Philippines,PH,PHL,Asia,South-eastern Asia,104918090,21.6,0.681799,68.340,11.733020,9.326870,8395.094758,Philippines,-80639361.0,-0.768593


In [84]:
#evidenzio solamente la differenza del denaro prestato e ricevuto, rispetto alla popolazione, della nazione che ha il valore più elevato:
statistiche_unite[statistiche_unite['differenza_popolazione']==statistiche_unite['differenza_popolazione'].max()]

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,differenza_nomissing,differenza_popolazione
47,Canada,CA,CAN,Americas,Northern America,36624199,9.4,0.920284,82.224,16.325,13.105115,42581.91433,Canada,68982039.0,1.88351


In [85]:
#Punto 11
#Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [86]:
#carico il dataset creato precedentemente:
statistiche_unite.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,differenza_nomissing,differenza_popolazione
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India,-6832668.0,-0.005102
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria,-2390881.0,-0.012525
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico,-14645955.0,-0.113391
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan,-19823359.0,-0.100618
4,Democratic Republic of the Congo,CD,COD,Africa,Middle Africa,81339988,63.0,0.435465,59.057,9.750000,6.117153,679.631223,The Democratic Republic of the Congo,-16355228.0,-0.201072


In [87]:
#calcolo il totale della popolazione sotto la soglia di povertà:
statistiche_unite['popolazione_sotto_soglia'] = (statistiche_unite['population'] * statistiche_unite['population_below_poverty_line']) / 100
statistiche_unite.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,differenza_nomissing,differenza_popolazione,popolazione_sotto_soglia
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India,-6832668.0,-0.005102,2.932804e+08
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria,-2390881.0,-0.012525,1.336204e+08
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico,-14645955.0,-0.113391,5.967343e+07
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan,-19823359.0,-0.100618,5.811971e+07
4,Democratic Republic of the Congo,CD,COD,Africa,Middle Africa,81339988,63.0,0.435465,59.057,9.750000,6.117153,679.631223,The Democratic Republic of the Congo,-16355228.0,-0.201072,5.124419e+07


In [88]:
#calcolo la differenza con la popolazione totale, in modo tale da ottenere la popolazione sopra la soglia:
statistiche_unite['popolazione_sopra_soglia'] = statistiche_unite['population'] - statistiche_unite['popolazione_sotto_soglia']
statistiche_unite.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,differenza_nomissing,differenza_popolazione,popolazione_sotto_soglia,popolazione_sopra_soglia
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India,-6832668.0,-0.005102,2.932804e+08,1.045900e+09
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria,-2390881.0,-0.012525,1.336204e+08,5.726589e+07
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico,-14645955.0,-0.113391,5.967343e+07,6.948984e+07
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan,-19823359.0,-0.100618,5.811971e+07,1.388962e+08
4,Democratic Republic of the Congo,CD,COD,Africa,Middle Africa,81339988,63.0,0.435465,59.057,9.750000,6.117153,679.631223,The Democratic Republic of the Congo,-16355228.0,-0.201072,5.124419e+07,3.009580e+07


In [89]:
#calcolo il rapporto tra la percentuale di loan per persona ed il totale di persone sopra la soglia di povertà:
statistiche_unite['differenza_povertà'] = statistiche_unite['differenza_popolazione'] / statistiche_unite['popolazione_sopra_soglia']
statistiche_unite.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,differenza_nomissing,differenza_popolazione,popolazione_sotto_soglia,popolazione_sopra_soglia,differenza_povertà
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India,-6832668.0,-0.005102,2.932804e+08,1.045900e+09,-4.878219e-12
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria,-2390881.0,-0.012525,1.336204e+08,5.726589e+07,-2.187193e-10
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico,-14645955.0,-0.113391,5.967343e+07,6.948984e+07,-1.631764e-09
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan,-19823359.0,-0.100618,5.811971e+07,1.388962e+08,-7.244115e-10
4,Democratic Republic of the Congo,CD,COD,Africa,Middle Africa,81339988,63.0,0.435465,59.057,9.750000,6.117153,679.631223,The Democratic Republic of the Congo,-16355228.0,-0.201072,5.124419e+07,3.009580e+07,-6.681080e-09


In [90]:
#trovo quindi il valore della nazione che presenta la maggior percentuale di differenza di povertà:
max_povertà = statistiche_unite['differenza_povertà'].max()
statistiche_unite[statistiche_unite['differenza_povertà'] == max_povertà]

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,differenza_nomissing,differenza_popolazione,popolazione_sotto_soglia,popolazione_sopra_soglia,differenza_povertà
47,Canada,CA,CAN,Americas,Northern America,36624199,9.4,0.920284,82.224,16.325,13.105115,42581.91433,Canada,68982039.0,1.88351,3442674.706,3.318152e+07,5.676381e-08


In [85]:
#Punto 12
#For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [91]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,disburse_time_date,planned_expiration_time_date,duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2,1,NaN,female,true,irregular,field_partner,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,53 days 19:30:06
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2,1,NaN,female,true,monthly,field_partner,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,96 days 14:25:07
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,37 days 13:10:05
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2,1,user_favorite,female,true,monthly,field_partner,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,34 days 19:10:02
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,57 days 22:10:02


In [92]:
#del dataset legato ai prestiti, mi interessano solo alcune informazioni, che vado a raccogliere in un dataset unico:
loans_utili = loans[['loan_id', 'disburse_time_date', 'planned_expiration_time_date', 'loan_amount']]
loans_utili.head()

,loan_id,disburse_time_date,planned_expiration_time_date,loan_amount
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,125.0
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,400.0
2,658010,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,400.0
3,659347,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,625.0
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,425.0


In [93]:
#come visto in precedenza, sono presenti dei dati mancanti:
loans_utili.isnull().sum()

loan_id                              0
disburse_time_date                2813
planned_expiration_time_date    371834
loan_amount                          0
dtype: int64

In [94]:
#procediamo con l'eliminare i dati mancanti:
loans_utili_no_missing = loans_utili.dropna()

In [95]:
#controlliamo che effettivamente non siano presenti dati mancanti:
loans_utili_no_missing.isnull().sum()

loan_id                         0
disburse_time_date              0
planned_expiration_time_date    0
loan_amount                     0
dtype: int64

In [96]:
#decido di eliminare il fuso orario che e renderli "normalizzati", in modo tale da non avere problemi di date in un momento successivo:
loans_utili_no_missing["disburse_time_date"].dt.tz_localize(None)
loans_utili_no_missing["planned_expiration_time_date"].dt.tz_localize(None)

0         2014-02-14 03:30:06
1         2014-03-26 22:25:07
2         2014-02-15 21:10:05
3         2014-02-21 03:10:02
4         2014-02-13 06:10:02
                  ...        
1419602   2016-01-02 01:00:03
1419603   2016-01-02 16:40:07
1419604   2016-01-03 22:20:04
1419605   2016-01-05 08:50:02
1419606   2016-01-03 20:50:06
Name: planned_expiration_time_date, Length: 1044962, dtype: datetime64[ns]

In [97]:
#quello che voglio realizzare, non è altro che utilizzare le date e gli anni di riferimento, per trovare quanti giorni ci sono per anno, successivamente calcolare dell'ammontare totale un ammontare giornaliero per anno: 
#trovo l'anno per quanto riguarda disburse_year:
loans_utili_no_missing["disburse_year"] = loans_utili_no_missing["disburse_time_date"].dt.year
#trovo l'anno per quanto riguarda planned_expiration_year:
loans_utili_no_missing["planned_expiration_year"] = loans_utili_no_missing["planned_expiration_time_date"].dt.year
#calcolo quindi la differenza degli anni tra i due tempi di prestito e restituzione:
loans_utili_no_missing['differenza_anni'] = loans_utili_no_missing['planned_expiration_year'] - loans_utili_no_missing['disburse_year']
loans_utili_no_missing.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

,loan_id,disburse_time_date,planned_expiration_time_date,loan_amount,disburse_year,planned_expiration_year,differenza_anni
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,125.0,2013,2014,1
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,400.0,2013,2014,1
2,658010,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,400.0,2014,2014,0
3,659347,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,625.0,2014,2014,0
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,425.0,2013,2014,1


In [98]:
#divido il dataset in due tipi, in particolare so che ci sono alcuni anni che sono zero, questo perchè sia il disburse che il planned_expiration_time, avvengono nello stesso anno:
loans_nodifferenzaanni = loans_utili_no_missing[loans_utili_no_missing['differenza_anni'] == 0]
loans_nodifferenzaanni.head()

,loan_id,disburse_time_date,planned_expiration_time_date,loan_amount,disburse_year,planned_expiration_year,differenza_anni
2,658010,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,400.0,2014,2014,0
3,659347,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,625.0,2014,2014,0
5,659605,2014-01-15 08:00:00+00:00,2014-02-20 02:30:07+00:00,350.0,2014,2014,0
6,660240,2014-01-20 08:00:00+00:00,2014-02-21 07:50:11+00:00,125.0,2014,2014,0
7,661601,2014-01-10 08:00:00+00:00,2014-02-25 09:50:03+00:00,1600.0,2014,2014,0


In [99]:
#trovo il dataset, invece, dove la differenza degli anni non è zero, e quindi potrebbe intercorrere anche più anni:
loans_differenzaanni = loans_utili_no_missing[loans_utili_no_missing['differenza_anni'] != 0]
loans_differenzaanni.head()

,loan_id,disburse_time_date,planned_expiration_time_date,loan_amount,disburse_year,planned_expiration_year,differenza_anni
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,125.0,2013,2014,1
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,400.0,2013,2014,1
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,425.0,2013,2014,1
12,660363,2013-12-23 08:00:00+00:00,2014-02-21 17:10:02+00:00,1175.0,2013,2014,1
13,661165,2013-12-26 08:00:00+00:00,2014-03-26 22:24:34+00:00,300.0,2013,2014,1


In [112]:
#Gli anni quindi so che vanno dal 2011 al 2018, si tratta di un range di 7 anni di prestiti:
print(loans_utili_no_missing['disburse_year'].min(), loans_utili_no_missing['disburse_year'].max(), loans_utili_no_missing['planned_expiration_year'].min(), loans_utili_no_missing['planned_expiration_year'].max())

2011 2018 2012 2018


In [102]:
#voglio capire quanti giorni ci sono fino alla fine dell'anno nel caso del disburse e quandi giorni sono passati dall'inizio dell'anno nel caso del planned_expiration_time:
#mi affido alla libreria legata alle timeseries, per andare a trovare queste informazioni:
from pandas.tseries import offsets
#calcolo quindi quale sarebbe l'ultimo giorno dell'anno di riferimento del disburse_time:
loans_differenzaanni['fine_anno'] = loans_differenzaanni.disburse_time_date.apply(lambda i : i + pd.offsets.YearEnd())
loans_differenzaanni.head(10)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,loan_id,disburse_time_date,planned_expiration_time_date,loan_amount,disburse_year,planned_expiration_year,differenza_anni,fine_anno
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,125.0,2013,2014,1,2013-12-31 08:00:00+00:00
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,400.0,2013,2014,1,2013-12-31 08:00:00+00:00
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,425.0,2013,2014,1,2013-12-31 08:00:00+00:00
12,660363,2013-12-23 08:00:00+00:00,2014-02-21 17:10:02+00:00,1175.0,2013,2014,1,2013-12-31 08:00:00+00:00
13,661165,2013-12-26 08:00:00+00:00,2014-03-26 22:24:34+00:00,300.0,2013,2014,1,2013-12-31 08:00:00+00:00
71,808452,2014-11-06 08:00:00+00:00,2015-01-18 17:50:04+00:00,225.0,2014,2015,1,2014-12-31 08:00:00+00:00
74,1204329,2016-11-18 08:00:00+00:00,2017-01-14 14:20:02+00:00,725.0,2016,2017,1,2016-12-31 08:00:00+00:00
75,1212312,2016-12-07 08:00:00+00:00,2017-01-28 03:20:03+00:00,3750.0,2016,2017,1,2016-12-31 08:00:00+00:00
79,659111,2013-12-19 08:00:00+00:00,2014-02-19 16:50:02+00:00,875.0,2013,2014,1,2013-12-31 08:00:00+00:00
82,661912,2013-12-28 08:00:00+00:00,2014-02-25 13:20:04+00:00,625.0,2013,2014,1,2013-12-31 08:00:00+00:00


In [112]:
#Ora devo vedere quale sarebbe il primo giorno rispetto all'anno di riferimento del planned_expiration_time:
loans_differenzaanni['inizio_anno'] = loans_differenzaanni.planned_expiration_time_date.apply(lambda i : i - pd.offsets.YearBegin())
loans_differenzaanni.head(10)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,loan_id,disburse_time_date,planned_expiration_time_date,loan_amount,disburse_year,planned_expiration_year,differenza_anni,fine_anno,inizio_anno,differenza_giorni_alla_fine,giorni_differenza_totale
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,125.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 03:30:06+00:00,9,53
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,400.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 22:25:07+00:00,11,96
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,425.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 06:10:02+00:00,14,57
12,660363,2013-12-23 08:00:00+00:00,2014-02-21 17:10:02+00:00,1175.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 17:10:02+00:00,8,60
13,661165,2013-12-26 08:00:00+00:00,2014-03-26 22:24:34+00:00,300.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 22:24:34+00:00,5,90
71,808452,2014-11-06 08:00:00+00:00,2015-01-18 17:50:04+00:00,225.0,2014,2015,1,2014-12-31 08:00:00+00:00,2015-01-01 17:50:04+00:00,55,73
74,1204329,2016-11-18 08:00:00+00:00,2017-01-14 14:20:02+00:00,725.0,2016,2017,1,2016-12-31 08:00:00+00:00,2017-01-01 14:20:02+00:00,43,57
75,1212312,2016-12-07 08:00:00+00:00,2017-01-28 03:20:03+00:00,3750.0,2016,2017,1,2016-12-31 08:00:00+00:00,2017-01-01 03:20:03+00:00,24,51
79,659111,2013-12-19 08:00:00+00:00,2014-02-19 16:50:02+00:00,875.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 16:50:02+00:00,12,62
82,661912,2013-12-28 08:00:00+00:00,2014-02-25 13:20:04+00:00,625.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 13:20:04+00:00,3,59


In [114]:
#calcolo quindi quanti giorni mancano dalla data del disburse_time alla data di fine anno:
loans_differenzaanni['differenza_giorni_alla_fine'] = (loans_differenzaanni['fine_anno'] - loans_differenzaanni['disburse_time_date']).dt.days
loans_differenzaanni.head(10)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,loan_id,disburse_time_date,planned_expiration_time_date,loan_amount,disburse_year,planned_expiration_year,differenza_anni,fine_anno,inizio_anno,differenza_giorni_alla_fine,giorni_differenza_totale
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,125.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 03:30:06+00:00,9,53
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,400.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 22:25:07+00:00,11,96
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,425.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 06:10:02+00:00,14,57
12,660363,2013-12-23 08:00:00+00:00,2014-02-21 17:10:02+00:00,1175.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 17:10:02+00:00,8,60
13,661165,2013-12-26 08:00:00+00:00,2014-03-26 22:24:34+00:00,300.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 22:24:34+00:00,5,90
71,808452,2014-11-06 08:00:00+00:00,2015-01-18 17:50:04+00:00,225.0,2014,2015,1,2014-12-31 08:00:00+00:00,2015-01-01 17:50:04+00:00,55,73
74,1204329,2016-11-18 08:00:00+00:00,2017-01-14 14:20:02+00:00,725.0,2016,2017,1,2016-12-31 08:00:00+00:00,2017-01-01 14:20:02+00:00,43,57
75,1212312,2016-12-07 08:00:00+00:00,2017-01-28 03:20:03+00:00,3750.0,2016,2017,1,2016-12-31 08:00:00+00:00,2017-01-01 03:20:03+00:00,24,51
79,659111,2013-12-19 08:00:00+00:00,2014-02-19 16:50:02+00:00,875.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 16:50:02+00:00,12,62
82,661912,2013-12-28 08:00:00+00:00,2014-02-25 13:20:04+00:00,625.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 13:20:04+00:00,3,59


In [119]:
#calcolo la differenza dal giorno del planned_expiration_time, con la data di inizio dell'anno:
loans_differenzaanni['differenza_giorni_inizio'] = (loans_differenzaanni['planned_expiration_time_date'] - loans_differenzaanni['inizio_anno']).dt.days
loans_utili_no_missing.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,loan_id,disburse_time_date,planned_expiration_time_date,loan_amount,disburse_year,planned_expiration_year,differenza_anni
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,125.0,2013,2014,1
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,400.0,2013,2014,1
2,658010,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,400.0,2014,2014,0
3,659347,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,625.0,2014,2014,0
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,425.0,2013,2014,1


In [110]:
#calcolo quanti giorni sono presenti come differenza tra la data di planned_expiration alla data di disburse:
loans_differenzaanni['giorni_differenza_totale'] = (loans_differenzaanni['planned_expiration_time_date'] - loans_differenzaanni['disburse_time_date']).dt.days
loans_differenzaanni.head(10)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,loan_id,disburse_time_date,planned_expiration_time_date,loan_amount,disburse_year,planned_expiration_year,differenza_anni,fine_anno,inizio_anno,differenza_giorni_alla_fine,giorni_differenza_totale
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,125.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 03:30:06+00:00,9,53
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,400.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 22:25:07+00:00,11,96
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,425.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 06:10:02+00:00,14,57
12,660363,2013-12-23 08:00:00+00:00,2014-02-21 17:10:02+00:00,1175.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 17:10:02+00:00,8,60
13,661165,2013-12-26 08:00:00+00:00,2014-03-26 22:24:34+00:00,300.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 22:24:34+00:00,5,90
71,808452,2014-11-06 08:00:00+00:00,2015-01-18 17:50:04+00:00,225.0,2014,2015,1,2014-12-31 08:00:00+00:00,2015-01-01 17:50:04+00:00,55,73
74,1204329,2016-11-18 08:00:00+00:00,2017-01-14 14:20:02+00:00,725.0,2016,2017,1,2016-12-31 08:00:00+00:00,2017-01-01 14:20:02+00:00,43,57
75,1212312,2016-12-07 08:00:00+00:00,2017-01-28 03:20:03+00:00,3750.0,2016,2017,1,2016-12-31 08:00:00+00:00,2017-01-01 03:20:03+00:00,24,51
79,659111,2013-12-19 08:00:00+00:00,2014-02-19 16:50:02+00:00,875.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 16:50:02+00:00,12,62
82,661912,2013-12-28 08:00:00+00:00,2014-02-25 13:20:04+00:00,625.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 13:20:04+00:00,3,59


In [120]:
#calcolo i giorni di differenza totale:
loans_differenzaanni['giorni_di_mezzo'] = loans_differenzaanni['giorni_differenza_totale'] - loans_differenzaanni['differenza_giorni_inizio'] - loans_differenzaanni['differenza_giorni_alla_fine']
loans_differenzaanni

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,loan_id,disburse_time_date,planned_expiration_time_date,loan_amount,disburse_year,planned_expiration_year,differenza_anni,fine_anno,inizio_anno,differenza_giorni_alla_fine,giorni_differenza_totale,differenza_giorni_inizio,giorni_di_mezzo
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,125.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 03:30:06+00:00,9,53,44,0
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,400.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 22:25:07+00:00,11,96,84,1
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,425.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 06:10:02+00:00,14,57,43,0
12,660363,2013-12-23 08:00:00+00:00,2014-02-21 17:10:02+00:00,1175.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 17:10:02+00:00,8,60,51,1
13,661165,2013-12-26 08:00:00+00:00,2014-03-26 22:24:34+00:00,300.0,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 22:24:34+00:00,5,90,84,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419602,988180,2015-11-23 08:00:00+00:00,2016-01-02 01:00:03+00:00,400.0,2015,2016,1,2015-12-31 08:00:00+00:00,2016-01-01 01:00:03+00:00,38,39,1,0
1419603,988213,2015-11-24 08:00:00+00:00,2016-01-02 16:40:07+00:00,300.0,2015,2016,1,2015-12-31 08:00:00+00:00,2016-01-01 16:40:07+00:00,37,39,1,1
1419604,989109,2015-11-13 08:00:00+00:00,2016-01-03 22:20:04+00:00,2425.0,2015,2016,1,2015-12-31 08:00:00+00:00,2016-01-01 22:20:04+00:00,48,51,2,1
1419605,989143,2015-11-03 08:00:00+00:00,2016-01-05 08:50:02+00:00,100.0,2015,2016,1,2015-12-31 08:00:00+00:00,2016-01-01 08:50:02+00:00,58,63,4,1


In [121]:
#quello che voglio utilizzare è una funzione che mi permetta per ogni anno, di avere l'ammontare unitario dei prestiti in base ai giorni che sono presenti, per poi raccogliere tutto sotto forma di anno come somma di tutti gli importi.
#il mio ragionamento, quindi, si è basato sul creare una lista, nella quale inserire due diverse voci, che verranno create in seguito da una funzione. Queste voci saranno anno di riferimento, e ammontare di denaro del prestito, in base ai giorni riferiti ad un determinato prestito. Utilizzerò una funzione per fare ciò, nella quale per ogni anno calcolo l'ammontare del prestito per il numero di giorni dalla data di disburse alla fine dell'anno, e dalla data di expiration alla data di inizio anno. Per poi dividere per 
lista_per_gli_anni = []
giorni_in_un_anno = 365

In [122]:
#creo la funzione, se nel caso in cui l'anno dovesse combaciare con quello dell'expiration, ovviamente l'ammontare dovrà bloccarsi e restituire l'anno e l'ammontare dell'anno di riferimento.
#Inoltre, con questa funzione che poi applicheremo al dataset di riferimento degli anni diversi dallo stesso, prima sarà effettuato il conteggio dell'ammontare per giorni per il disburse_time, poi per l'expiration:
def function(dataframe):
    global lista_per_gli_anni
    disburse_year = dataframe['disburse_year']
    planned_expiration_year = dataframe['planned_expiration_year']
    for i in range(disburse_year, planned_expiration_year, 1):
        x_year_ammontare = (dataframe['loan_amount']*dataframe['differenza_giorni_alla_fine']) / dataframe['giorni_differenza_totale']
        risultato = {'prestito_ammontare': x_year_ammontare, 'year':disburse_year}
        lista_per_gli_anni.append(risultato)
        dataframe['differenza_giorni_alla_fine'] = giorni_in_un_anno
        disburse_year = disburse_year+1
        x_year_ammontare = (dataframe['loan_amount']*dataframe['differenza_giorni_inizio']) / dataframe['giorni_differenza_totale']
        anni = dataframe['planned_expiration_year']
        risultato = {'prestito_ammontare':x_year_ammontare, 'year':anni}
        lista_per_gli_anni.append(risultato)
    return

In [124]:
#applico la funzione creata in precedenza al dataset dove gli anni sono diversi gli uni dagli altri.
loans_differenzaanni.apply(lambda i : function(i), axis = 'columns')
#I risultatti mostrano None perchè i valori sono posizionati nella lista creata in precedenza:

0          None
1          None
4          None
12         None
13         None
           ... 
1419602    None
1419603    None
1419604    None
1419605    None
1419606    None
Length: 152157, dtype: object

In [125]:
#i risultati sono stati posizionati nella lista che ho creato in precedenza
lista_per_gli_anni
#creo un dataframe, per rappresentare meglio i dati, che poi mi saranno utili per l'operazione finale con il dataset dove l'anno di distribution ed expiration combaciano:
dataframe_anni_prestiti = pd.DataFrame(lista_per_gli_anni)
dataframe_anni_prestiti.head()

,prestito_ammontare,year
0,21.226415,2013
1,103.773585,2014
2,45.833333,2013
3,350.000000,2014
4,104.385965,2013


In [126]:
#Mostro i risultati in maniera intera, per essere più chiari e comprensibili:
data_frame_anni_prestiti_diff = round(dataframe_anni_prestiti.groupby('year').sum(),0)
data_frame_anni_prestiti_diff.head(10)

,prestito_ammontare
year,
2011,1689354.0
2012,20320158.0
2013,58206604.0
2014,77097951.0
2015,59185704.0
2016,50047329.0
2017,56057343.0
2018,17213054.0


In [154]:
loans_nodifferenzaanni.head()

,loan_id,disburse_time_date,planned_expiration_time_date,loan_amount,disburse_year,planned_expiration_year,differenza_anni
2,658010,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,400.0,2014,2014,0
3,659347,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,625.0,2014,2014,0
5,659605,2014-01-15 08:00:00+00:00,2014-02-20 02:30:07+00:00,350.0,2014,2014,0
6,660240,2014-01-20 08:00:00+00:00,2014-02-21 07:50:11+00:00,125.0,2014,2014,0
7,661601,2014-01-10 08:00:00+00:00,2014-02-25 09:50:03+00:00,1600.0,2014,2014,0


In [127]:
#scegliamo quindi come anno uno qualsiasi tra distribution ed expiration, visto che sono nello stesso anno come decorrenza:
loans_nodifferenzaanni['year'] = loans_nodifferenzaanni['disburse_year']
#eseguo un groupby per gli anni, in modo da avere l'ammontare totale dell'anno di riferimento del prestito:
dataframe_prestiti_anni_stessoanno = loans_nodifferenzaanni.groupby('disburse_year')['loan_amount'].sum()
dataframe_prestiti_anni_stessoanno.head(10)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


disburse_year
2012    104138000.0
2013     99878775.0
2014    123149050.0
2015    135779450.0
2016    138206675.0
2017    151284475.0
2018       710425.0
Name: loan_amount, dtype: float64

In [128]:
#trasformo in un dataframe, in modo tale da poter successivamente unire e rappresentare pià chiaramente con il dataframe dove gli anni non combaciavano:
dataframe_prestiti_anni_stessoanno_df = pd.DataFrame(dataframe_prestiti_anni_stessoanno).reset_index()
dataframe_prestiti_anni_stessoanno_df.head(10)

,disburse_year,loan_amount
0,2012,104138000.0
1,2013,99878775.0
2,2014,123149050.0
3,2015,135779450.0
4,2016,138206675.0
5,2017,151284475.0
6,2018,710425.0


In [129]:
#Unico i due dataset, in modo da avere una sola tabella con le due colonne di ammontare totale degli anni dei prestiti:
ammontare_totale_per_anni = pd.merge(dataframe_prestiti_anni_stessoanno_df, data_frame_anni_prestiti_diff, left_on = "disburse_year", right_on = 'year')
ammontare_totale_per_anni.head()

,disburse_year,loan_amount,prestito_ammontare
0,2012,104138000.0,20320158.0
1,2013,99878775.0,58206604.0
2,2014,123149050.0,77097951.0
3,2015,135779450.0,59185704.0
4,2016,138206675.0,50047329.0


In [130]:
#raggruppo per anno, per avere per ogni singolo anno, l'ammontare dei prestiti sia da parte di prestiti dello stesso anno che di anni diversi:
ammontare_totale_per_anni_def = ammontare_totale_per_anni.groupby('disburse_year')[['loan_amount', 'prestito_ammontare']].sum()
ammontare_totale_per_anni.head()

,disburse_year,loan_amount,prestito_ammontare
0,2012,104138000.0,20320158.0
1,2013,99878775.0,58206604.0
2,2014,123149050.0,77097951.0
3,2015,135779450.0,59185704.0
4,2016,138206675.0,50047329.0


In [136]:
#calcolo quindi l'ammontare di prestito totale per anni, sia nel caso di prestiti dello stesso anno, che di prestiti di anni diversi:
ammontare_totale_per_anni_def['totale_ammontare'] = ammontare_totale_per_anni_def['loan_amount'] + ammontare_totale_per_anni_def['prestito_ammontare']
ammontare_totale_per_anni_def1 = ammontare_totale_per_anni_def.reset_index()
ammontare_totale_per_anni_def1

,disburse_year,loan_amount,prestito_ammontare,totale_ammontare
0,2012,104138000.0,20320158.0,124458158.0
1,2013,99878775.0,58206604.0,158085379.0
2,2014,123149050.0,77097951.0,200247001.0
3,2015,135779450.0,59185704.0,194965154.0
4,2016,138206675.0,50047329.0,188254004.0
5,2017,151284475.0,56057343.0,207341818.0
6,2018,710425.0,17213054.0,17923479.0


In [139]:
#seleziono solo le colonne che mi sono utili per la task del mio problema, ossia quindi l'anno e l'ammontare totale:
ammontare_totale_per_anni_def_totale = ammontare_totale_per_anni_def1[['disburse_year', 'totale_ammontare']]
ammontare_totale_per_anni_def_totale.rename(columns={'disburse_year':'ANNO'})

,ANNO,totale_ammontare
0,2012,124458158.0
1,2013,158085379.0
2,2014,200247001.0
3,2015,194965154.0
4,2016,188254004.0
5,2017,207341818.0
6,2018,17923479.0
